# 🙇 Searching Errors in Dataset

## ☝️Task Description
This test task presents a sample of high-frequency data containing trades and updates to the limit order book. We have carefully concealed a substantial number of realistic errors in it, and your task is to identify as many errors as possible.

You don't need to show how you searched for errors, create plots, or perform any other analytics.
You also don't need to indicate the row numbers where you found errors.
The only thing required is a brief description (one line is enough) of the nature of each error found.
Task score - percentage of error types found.

## 💻 Get ready data for analysis

In [1]:
# upload dataset
import pandas as pd

# read file
df = pd.read_feather("test.feather").reset_index(drop=True)

# overview dataset
df.head()


,type,msgSeqNum,exchHostTime,adapterTime,px_buy_1,amt_buy_1,px_buy_2,amt_buy_2,px_buy_3,amt_buy_3,...,px_sell_8,amt_sell_8,px_sell_9,amt_sell_9,px_sell_10,amt_sell_10,trade_px,trade_amt,trade_cnt,moreTradesInBatch
0,4,173525833,1596240037000000000,1596240037431704679,11357.99,20.199,11357.67,0.169,11357.5,0.062,...,11358.82,0.001,12494.7,0.063,13744.17,0.111,11358.25,0.175,0,0.0
1,4,173525834,1596240037000000000,1596240037431749155,11357.99,20.199,11357.67,0.169,11357.5,0.062,...,11358.82,0.001,12494.7,0.063,13744.17,0.111,11358.26,2.000,0,0.0
2,4,122537841,1596240037000000000,1596240037431776197,11357.99,20.199,11357.67,0.169,11357.5,0.062,...,11358.82,0.001,12494.7,0.063,13744.17,0.111,11358.25,0.175,1,0.0
3,4,122537842,1596240037000000000,1596240037431782097,11357.99,20.199,11357.67,0.169,11357.5,0.062,...,11358.82,0.001,12494.7,0.063,13744.17,0.111,11358.26,2.000,1,0.0
4,4,173525835,1596240037000000000,1596240037431794710,11357.99,20.199,11357.67,0.169,11357.5,0.062,...,11358.82,0.001,12494.7,0.063,13744.17,0.111,11358.33,0.004,0,0.0


In [2]:
# overview data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40374 entries, 0 to 40373
Data columns (total 48 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   type               40374 non-null  int64  
 1   msgSeqNum          40374 non-null  int64  
 2   exchHostTime       40374 non-null  int64  
 3   adapterTime        40374 non-null  int64  
 4   px_buy_1           40373 non-null  float64
 5   amt_buy_1          40373 non-null  float64
 6   px_buy_2           40374 non-null  float64
 7   amt_buy_2          40373 non-null  float64
 8   px_buy_3           40374 non-null  float64
 9   amt_buy_3          40373 non-null  float64
 10  px_buy_4           40374 non-null  float64
 11  amt_buy_4          40373 non-null  float64
 12  px_buy_5           40374 non-null  float64
 13  amt_buy_5          40373 non-null  float64
 14  px_buy_6           40374 non-null  float64
 15  amt_buy_6          40373 non-null  float64
 16  px_buy_7           403

**Dataset description and column meanings**

1. System and timestamp fields
- **type** — message type (event code).
- **msgSeqNum** — sequential message number.
- **exchHostTime** — event timestamp on the exchange side.
- **adapterTime** — event reception timestamp on the adapter side.

2. Order book — Buy orders (Bids)
- **px_buy_[1–10]** — bid price at levels 1–10 (1 — best).
- **amt_buy_[1–10]** — bid order volume at levels 1–10.

3. Order book — Sell orders (Asks)
- **px_sell_[1–10]** — ask price at levels 1–10 (1 — best).
- **amt_sell_[1–10]** — ask order volume at levels 1–10.

4. Trades
- **trade_px** — trade price.
- **trade_amt** — trade volume.
- **trade_cnt** — number of trades in this message.

5. Additional trade field
- **moreTradesInBatch** — flag/indicator of other trades present in the batch.


**adjust datatypes**

In [3]:
# Convert nanoseconds to datetime
df['exchHostTime_dt'] = pd.to_datetime(df['exchHostTime'], unit='ns')
df['adapterTime_dt'] = pd.to_datetime(df['adapterTime'], unit='ns')

# Difference between adapterTime and exchHostTime in seconds
df['time_diff_sec'] = (df['adapterTime'] - df['exchHostTime']) / 1e9

# Difference in milliseconds
df['time_diff_ms'] = df['time_diff_sec'] * 1000

# Display first 10 rows for verification
df.head()


,type,msgSeqNum,exchHostTime,adapterTime,px_buy_1,amt_buy_1,px_buy_2,amt_buy_2,px_buy_3,amt_buy_3,...,px_sell_10,amt_sell_10,trade_px,trade_amt,trade_cnt,moreTradesInBatch,exchHostTime_dt,adapterTime_dt,time_diff_sec,time_diff_ms
0,4,173525833,1596240037000000000,1596240037431704679,11357.99,20.199,11357.67,0.169,11357.5,0.062,...,13744.17,0.111,11358.25,0.175,0,0.0,2020-08-01 00:00:37,2020-08-01 00:00:37.431704679,0.431705,431.704679
1,4,173525834,1596240037000000000,1596240037431749155,11357.99,20.199,11357.67,0.169,11357.5,0.062,...,13744.17,0.111,11358.26,2.000,0,0.0,2020-08-01 00:00:37,2020-08-01 00:00:37.431749155,0.431749,431.749155
2,4,122537841,1596240037000000000,1596240037431776197,11357.99,20.199,11357.67,0.169,11357.5,0.062,...,13744.17,0.111,11358.25,0.175,1,0.0,2020-08-01 00:00:37,2020-08-01 00:00:37.431776197,0.431776,431.776197
3,4,122537842,1596240037000000000,1596240037431782097,11357.99,20.199,11357.67,0.169,11357.5,0.062,...,13744.17,0.111,11358.26,2.000,1,0.0,2020-08-01 00:00:37,2020-08-01 00:00:37.431782097,0.431782,431.782097
4,4,173525835,1596240037000000000,1596240037431794710,11357.99,20.199,11357.67,0.169,11357.5,0.062,...,13744.17,0.111,11358.33,0.004,0,0.0,2020-08-01 00:00:37,2020-08-01 00:00:37.431794710,0.431795,431.794710


## 👀 Looking for data errors

In [4]:
# error number 1: px_sell_7 as object not float
df['px_sell_7'] = df['px_sell_7'].astype('float')

In [5]:
# error number 2: null values in dataset, the most big ratio in column "moreTradesInBatch"
df[df.isna().any(axis=1)].shape

(30156, 52)

In [6]:
# Calculate the share of nulls in each column
null_share = df.isna().mean()

# Find the column with the maximum share of nulls
max_null_col = null_share.idxmax()
max_null_share = null_share.max()

print(f"Column with the most nulls: {max_null_col}")
print(f"Share of nulls: {max_null_share:.2%}")


Column with the most nulls: moreTradesInBatch
Share of nulls: 74.69%


In [7]:
# error number 3: duplicated rows in dataset
df[df.duplicated()].shape

(100, 52)

In [8]:
# error number 4: violations of msgSeqNum increment

# Sort the DataFrame by msgSeqNum
df_sorted = df.sort_values('msgSeqNum').reset_index(drop=True)

# Calculate the difference between consecutive messages
diffs = df_sorted['msgSeqNum'].diff()

# Find rows where the difference is not equal to 1
violations = df_sorted[diffs != 1]

print(f"Found {len(violations)} violations of msgSeqNum increment")
print(violations[['msgSeqNum']].head(10))


Found 30250 violations of msgSeqNum increment
      msgSeqNum
0     122537841
628   122538478
1602  122539898
1603  122539901
1639  122540700
1947  122541031
2144  122541235
2256  122547739
2853  122548340
3292  122548781


In [9]:
# error number 5: trade_cnt < 0
df[df['trade_cnt']<0].shape

(1, 52)

In [10]:
# error number 6: trade_amt < 0
df[df['trade_amt']<0].shape

(4937, 52)

In [11]:
# error number 7: trade_amt > 0 & trade_cnt = 0
df[(df['trade_amt'] > 0) & (df['trade_cnt'] == 0)].shape

(3190, 52)

In [12]:
# error number 8: bid-ask crossing
crossed = df[df['px_buy_1'] > df['px_sell_1']]
crossed[['px_buy_1', 'px_sell_1']]

,px_buy_1,px_sell_1
20651,1.137400e+10,11374.01


In [13]:
# error number 9: Violations of monotonicity in bid levels

bid_cols = [f'px_buy_{i}' for i in range(1, 11)]

# Create an empty list to store results
violations = []

# Iterate over each row
for idx, row in df.iterrows():
    for i in range(len(bid_cols) - 1):
        if row[bid_cols[i]] < row[bid_cols[i+1]]:
            violations.append({
                'index': idx,
                'level_1': bid_cols[i],
                'level_2': bid_cols[i+1],
                'value_1': row[bid_cols[i]],
                'value_2': row[bid_cols[i+1]]
            })

# Convert to DataFrame for easier analysis
violations_df = pd.DataFrame(violations)

print(f"Found {len(violations_df)} violations of monotonicity in bid levels.")
violations_df


Found 2 violations of monotonicity in bid levels.


,index,level_1,level_2,value_1,value_2
0,9511,px_buy_9,px_buy_10,-11384.15,11384.13
1,14299,px_buy_1,px_buy_2,0.00,11381.78


In [14]:
# error number 10: Violations of monotonicity in ask levels

ask_cols = [f'px_sell_{i}' for i in range(1, 11)]

# List to store violations
violations = []

# Check each row
for idx, row in df.iterrows():
    for i in range(len(ask_cols) - 1):
        if row[ask_cols[i]] > row[ask_cols[i+1]]:
            violations.append({
                'index': idx,
                'level_1': ask_cols[i],
                'level_2': ask_cols[i+1],
                'value_1': row[ask_cols[i]],
                'value_2': row[ask_cols[i+1]]
            })

# Convert to DataFrame for analysis
violations_df = pd.DataFrame(violations)

print(f"Found {len(violations_df)} violations of monotonicity in ask levels.")
violations_df.head(10)


Found 7 violations of monotonicity in ask levels.


,index,level_1,level_2,value_1,value_2
0,18538,px_sell_1,px_sell_2,11373.38,11373.37
1,18538,px_sell_2,px_sell_3,11373.37,11373.04
2,18538,px_sell_3,px_sell_4,11373.04,11373.00
3,18538,px_sell_4,px_sell_5,11373.00,11372.41
4,18538,px_sell_5,px_sell_6,11372.41,11372.36
5,18538,px_sell_6,px_sell_7,11372.36,11372.33
6,18538,px_sell_7,px_sell_8,11372.33,11372.27


In [15]:
# error number 11: adapterTime and exchHostTime (> 1e9 in the same units) — suspicious timestamps/synchronization.

# Threshold in nanoseconds
threshold_ns = 1e9  # 1 second

# Calculate the difference between adapterTime and exchHostTime
df['time_diff_ns'] = df['adapterTime'] - df['exchHostTime']

# Filter rows with large discrepancies
suspect_timestamps = df[df['time_diff_ns'].abs() > threshold_ns]

print(f"Found {len(suspect_timestamps)} cases of large time differences (> {threshold_ns} ns).")
suspect_timestamps.head()


Found 953 cases of large time differences (> 1000000000.0 ns).


,type,msgSeqNum,exchHostTime,adapterTime,px_buy_1,amt_buy_1,px_buy_2,amt_buy_2,px_buy_3,amt_buy_3,...,amt_sell_10,trade_px,trade_amt,trade_cnt,moreTradesInBatch,exchHostTime_dt,adapterTime_dt,time_diff_sec,time_diff_ms,time_diff_ns
165,6,45252675608,1596240037000000000,1596240038008012737,11360.28,0.212,11360.0,19.140,11359.87,1.154,...,1.006,0.00,0.000,0,NaN,2020-08-01 00:00:37,2020-08-01 00:00:38.008012737,1.008013,1008.012737,1008012737
166,6,45252675649,1596240037000000000,1596240038008275634,11360.35,1.914,11360.3,0.178,11360.28,2.000,...,1.006,0.00,0.000,0,NaN,2020-08-01 00:00:37,2020-08-01 00:00:38.008275634,1.008276,1008.275634,1008275634
167,6,45252675714,1596240037000000000,1596240038008437196,11360.35,1.914,11360.3,1.910,11360.28,2.000,...,0.003,0.00,0.000,0,NaN,2020-08-01 00:00:37,2020-08-01 00:00:38.008437196,1.008437,1008.437196,1008437196
168,4,173525903,1596240037000000000,1596240038008518051,11360.35,1.914,11360.3,1.910,11360.28,2.000,...,0.003,11360.35,-0.040,0,0.0,2020-08-01 00:00:37,2020-08-01 00:00:38.008518051,1.008518,1008.518051,1008518051
169,4,122537882,1596240037000000000,1596240038008543243,11360.35,1.914,11360.3,1.910,11360.28,2.000,...,0.003,11360.27,-0.431,1,0.0,2020-08-01 00:00:37,2020-08-01 00:00:38.008543243,1.008543,1008.543243,1008543243


## 👑 Outcome Summary

During the data quality review of the high-frequency trading dataset, multiple types of inconsistencies and anomalies were detected:

1) Data type mismatch:
The column px_sell_7 is stored as an object rather than a float, which may cause calculation errors and should be converted to the correct numeric type.

2) High proportion of null values:
Several columns contain missing data, with the highest null share found in moreTradesInBatch — 74.69%, indicating incomplete trade batching information.

3) Duplicate records:
The dataset contains exact duplicate rows, which may distort statistics and model results.

4) Message sequence violations:
The msgSeqNum field, which should increment by 1 between consecutive messages, shows breaks in continuity, suggesting dropped or out-of-order messages.

5) Negative trade counts:
Instances where trade_cnt < 0 were found, which is logically impossible and indicates data corruption or mis-entry.

6) Negative trade amounts:
Cases with trade_amt < 0 exist, suggesting incorrect volume data.

7) Inconsistent trade amount/count:
Rows where trade_amt > 0 but trade_cnt = 0 indicate that a trade volume is recorded despite no trades being reported.

8) Bid-ask crossing:
Some rows have the best bid price greater than the best ask price (px_buy_1 > px_sell_1), violating basic market microstructure rules.

9) Monotonicity violations in bid levels:
In certain rows, deeper bid levels have prices higher than better bid levels, which is inconsistent with order book structure.

10) Monotonicity violations in ask levels:
Similar violations occur in the ask side, where deeper ask levels have lower prices than better ask levels.

11) Suspicious timestamp discrepancies:
Large differences (> 1 second) between adapterTime and exchHostTime suggest possible clock synchronization issues or incorrect timestamp recording.